# December 17, 2022
https://adventofcode.com/2022/day/17

In [1]:
# Too low: 3132, 3133
# Too high: 5256

In [2]:
test = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"

In [3]:
fn = "data/17.txt"
with open(fn, "r") as file:
    puz = file.readline()

In [9]:
# Tower is oriented [x,y] with [0,0] being left/bottom
class Tower:
    def __init__(self, jets, width=7):
        self.width = width

        # track which gust comes next
        self.jets = jets
        self.jet_pos = 0

        # instantiate map with floor
        # self.map is recorded from bottom up
        self.map = [ ['-']*self.width ]
        self.height = 0
        
        # track current active block
        # 0Flat, Cross, 2Ell, 3Pipe, 4Tuba
        self.next = 0
        self.block_count = 0
        self.spawn_x = 2
        self.spawn_dy = 4
        self.new_block()



    def check(self, x, y):
       # print("Checking",x,y)
        # walls are closed
        if x < 0 or x >= self.width:
            return False

        # open above
        if y > self.height:
            return True

        # otherwise check map
       # print("Looking at map")
       # print(self.map[y][x])
        return  (self.map[y][x]=='.') # using 0 for blank

    def new_block(self):
       # print("new block!")
        #print(self.map)
        y = self.height + self.spawn_dy
        if self.next == 0:
            self.block = Flat(self.spawn_x, y, self)
        elif self.next == 1:
            self.block = Cross(self.spawn_x,y, self)
        elif self.next == 2:
            self.block = Ell(self.spawn_x,y, self)
        elif self.next == 3:
            self.block = Pipe(self.spawn_x,y, self)
        elif self.next == 4:
            self.block = Tuba(self.spawn_x,y, self)
        self.next = (self.next + 1) % 5
        self.block_count += 1

    def gust(self):
        jet = self.jets[self.jet_pos]
        if jet == "<":
            if self.block.check_left():
                self.block.x -= 1
        else: # jet == ">":
            if self.block.check_right():
                self.block.x += 1
        self.jet_pos = (self.jet_pos + 1) % len(self.jets)


    def drop(self):
        # return True if block dropped, False if it's stuck
        if self.block.check_down():
            self.block.y -= 1
            return True
        else:
            pixels = self.block.pixels()
            for xy in pixels:
                self.solidify(xy[0], xy[1], self.block.char)
            self.new_block()
            return False

    def solidify(self, x, y, char):
        # add to tower height as needed
        dy = y - self.height
        for i in range(dy):
            self.map += [ ['.']*self.width ]
            self.height += 1

        self.map[y][x] = char

    def __str__(self):
        addendum = []
        out = []
        dy = self.block.y + (self.block.height-1) - self.height
        pixels = self.block.pixels()
        for y in range(dy):
            addendum.append( ['.']*self.width )

        for xy in pixels:
            if xy[1] > self.height:
                # add to addendum
                y2 = xy[1]-1 - self.height
                addendum[y2][xy[0]] = '@'
            else:
                self.map[xy[1]][xy[0]] = '@'

        for line in addendum[::-1]:
            out.append( "".join(line) )
        
        for line in self.map[::-1]:
            out.append( "".join(line) )


        for xy in pixels:
            if xy[1] <= self.height:
                self.map[xy[1]][xy[0]] = '.'
                
        # reset the map's block spaces
        return "\n".join(out)

class Block:
    pass

class Flat(Block):
    # refpoint is left-most
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Flat"
        self.height = 1
        self.char = "A"

    def check_right(self):
        return self.tower.check(self.x+4, self.y)
    def check_left(self):
        return self.tower.check(self.x-1, self.y)
    def check_down(self):
        for x in range(self.x, self.x+4):
            if not self.tower.check(x, self.y-1):
                return False
        return True
    def pixels(self):
        return [ [self.x, self.y], [self.x+1, self.y], [self.x+2, self.y], [self.x+3, self.y] ]

class Cross(Block):
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Cross"
        self.height = 3
        self.char = "B"


    def check_right(self):
        return (self.tower.check(self.x+2, self.y)
                and self.tower.check(self.x+3, self.y+1)
                and self.tower.check(self.x+2, self.y+2))
    def check_left(self):
        return (self.tower.check(self.x, self.y)
                and self.tower.check(self.x-1, self.y+1)
                and self.tower.check(self.x, self.y+2))
    def check_down(self):
        return (self.tower.check(self.x, self.y)
                and self.tower.check(self.x+1, self.y-1)
                and self.tower.check(self.x+2, self.y))
    def pixels(self):
        return [ [self.x+1, self.y], [self.x, self.y+1], [self.x+1, self.y+1], [self.x+2, self.y+1], [self.x+1, self.y+2] ]

class Ell(Block):
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Ell"
        self.height = 3
        self.char = "C"

    def check_right(self):
        return (self.tower.check(self.x+3, self.y)
                and self.tower.check(self.x+3, self.y+1)
                and self.tower.check(self.x+3, self.y+2))
    def check_left(self):
        return (self.tower.check(self.x-1, self.y)
                and self.tower.check(self.x+1, self.y+1)
                and self.tower.check(self.x+1, self.y+2))
    def check_down(self):
        return (self.tower.check(self.x, self.y-1)
                and self.tower.check(self.x+1, self.y-1)
                and self.tower.check(self.x+2, self.y-1))
    def pixels(self):
        return [ [self.x, self.y], [self.x+1, self.y], [self.x+2, self.y], [self.x+2, self.y+1], [self.x+2, self.y+2] ]

class Pipe(Block):
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Pipe"
        self.height = 4
        self.char = "D"


    def check_right(self):
        return (self.tower.check(self.x+1, self.y)
                and self.tower.check(self.x+1, self.y+1)
                and self.tower.check(self.x+1, self.y+2)
                and self.tower.check(self.x+1, self.y+3))
    def check_left(self):
        return (self.tower.check(self.x-1, self.y)
                and self.tower.check(self.x-1, self.y+1)
                and self.tower.check(self.x-1, self.y+2)
                and self.tower.check(self.x-1, self.y+3))
    def check_down(self):
        return self.tower.check(self.x, self.y-1)
    def pixels(self):
        return [ [self.x, self.y], [self.x, self.y+1], [self.x, self.y+2], [self.x, self.y+3] ]

class Tuba(Block): # Two-buh-two
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Tuba"
        self.height = 2
        self.char = "E"


    def check_right(self):
        return (self.tower.check(self.x+2, self.y)
                and self.tower.check(self.x+2, self.y+1))
    def check_left(self):
        return (self.tower.check(self.x-1, self.y)
                and self.tower.check(self.x-1, self.y+1))
    def check_down(self):
        return (self.tower.check(self.x, self.y-1)
                and self.tower.check(self.x+1, self.y-1))
    def pixels(self):
        return [ [self.x, self.y], [self.x+1, self.y], [self.x, self.y+1], [self.x+1, self.y+1] ]


In [5]:
def play_game( tower, until=2023, verbose=0 ):
    while tower.block_count < until:
        #print(tower.jets[tower.jet_pos], tower.block.type)
        if verbose >= 2:
            print("\n", tower.jets[tower.jet_pos], tower.block.type)
        tower.gust()
        if verbose >= 4:
            print(tower)
        success = tower.drop()
        if (not success and verbose >=1) or verbose >= 3:
            print("\n", tower, "\n", sep="")

In [32]:
test_tower = Tower(test)
play_game(test_tower, 2023)
print(test_tower.height)

3068


In [10]:
puz_tower = Tower(puz)
play_game(puz_tower,2023)
print(puz_tower.height)

3181


In [35]:
print(puz_tower)

....@..
....@..
..@@@..
.......
.......
.......
....B..
...BBB.
....B..
...AAAA
..EEC..
..EEC..
..CCCB.
.D..BBB
.D...B.
.D.AAAA
.D....C
.EE...C
.EEBCCC
..BBB..
...B...
..AAAA.
..EE.C.
..EE.C.
.D.CCC.
.D.B...
.DBBB..
.D.B...
.D.AAAA
.D.EEC.
.D.EEC.
.DBCCC.
.BBB...
..B....
..AAAA.
..D.EE.
..D.EE.
..D.C..
..D.C..
..CCC..
...B...
..BBB..
...B...
.AAAA..
.EE....
.EE....
.D..C..
.D..C..
.DCCCB.
.D..BBB
.DAAAB.
.DC....
.DC.EE.
CCC.EE.
.EE..D.
.EE..D.
..D..D.
..D..D.
..D..B.
..D.BBB
..C..B.
..CAAAA
CCCB...
..BBB..
...B...
.AAAA..
....D..
....D..
....D..
....D..
EE..C..
EE..C..
.BCCC..
BBB....
.B..D..
AAAAD..
EE..D..
EE..D..
.B..C..
BBB.C..
.BCCC..
.AAAA..
....EEC
....EEC
....CCC
.....B.
....BBB
.AAAAB.
..D....
..D....
..D.C..
..D.C..
..CCC..
...B...
..BBB..
.D.BD..
.D..D..
.DEED..
.DEED..
AAAAC..
EE..C..
EECCC..
AAAA...
..EE...
..EEB..
..DBBB.
..D.B..
..D.C..
..D.C..
..CCC..
...B...
..BBB..
...B...
..AAAA.
..EE...
..EE...
...D...
...D...
..CD...
..CDB..
CCCBBB.
AAAAB..
...D...
...D...
.EEDC..


In [ ]:
puz_tower = Tower(puz)
play_game(puz_tower, 2023, verbose=5)
print(puz_tower)

In [8]:
puz_tower = Tower(puz)
play_game(puz_tower, 30, verbose=3)
print(puz_tower)


 > Flat

...@@@@
.......
.......
-------


 > Flat

...@@@@
.......
-------


 < Flat

..@@@@.
-------


 > Flat

...@...
..@@@..
...@...
.......
.......
.......
...AAAA
-------


 < Cross

..@....
.@@@...
..@....
.......
.......
...AAAA
-------


 < Cross

.@.....
@@@....
.@.....
.......
...AAAA
-------


 > Cross

..@....
.@@@...
..@....
...AAAA
-------


 > Cross

....@..
....@..
..@@@..
.......
.......
.......
...B...
..BBB..
...B...
...AAAA
-------


 < Ell

...@...
...@...
.@@@...
.......
.......
...B...
..BBB..
...B...
...AAAA
-------


 < Ell

..@....
..@....
@@@....
.......
...B...
..BBB..
...B...
...AAAA
-------


 < Ell

..@....
..@....
@@@....
...B...
..BBB..
...B...
...AAAA
-------


 > Ell

..@....
..@....
..@....
..@....
.......
.......
.......
...C...
...C...
.CCC...
...B...
..BBB..
...B...
...AAAA
-------


 > Pipe

...@...
...@...
...@...
...@...
.......
.......
...C...
...C...
.CCC...
...B...
..BBB..
...B...
...AAAA
-------


 > Pipe

....@..
....@..
....@..
....@..

In [41]:
puz_tower.map

[['-', '-', '-', '-', '-', '-', '-'],
 ['.', '.', '.', 'A', 'A', 'A', 'A'],
 ['.', '.', '.', 'B', '.', 'D', '.'],
 ['.', '.', 'B', 'B', 'B', 'D', '.'],
 ['.', '.', '.', 'B', '.', 'D', '.'],
 ['.', 'C', 'C', 'C', '.', 'D', '.'],
 ['E', 'E', '.', 'C', '.', '.', '.'],
 ['E', 'E', '.', 'C', '.', '.', '.'],
 ['.', '.', '.', 'A', 'A', 'A', 'A'],
 ['.', '.', '.', 'B', '.', '.', '.'],
 ['.', '.', 'B', 'B', 'B', '.', '.'],
 ['.', '.', '.', 'B', '.', '.', '.'],
 ['.', '.', 'C', 'C', 'C', '.', '.'],
 ['.', '.', '.', '.', 'C', '.', '.'],
 ['.', '.', '.', '.', 'C', '.', '.'],
 ['.', '.', '.', '.', 'D', '.', '.'],
 ['.', '.', '.', '.', 'D', '.', '.'],
 ['.', '.', '.', '.', 'D', '.', '.'],
 ['.', '.', '.', '.', 'D', '.', '.']]

In [39]:
str(puz_tower)

'....#..\n....#..\n....#..\n....#..\n....#..\n....#..\n..###..\n...#...\n..###..\n...#...\n...####\n##.#...\n##.#...\n.###.#.\n...#.#.\n..####.\n...#.#.\n...####\n#######'